In [1]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from torchvision import transforms
import os
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
import matplotlib.pyplot as plt

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Wrangling
Summary:
- Developed LightningDataModule to handle dataset and dataloader for train,val,test sets
- Apply appropriate transformation for augmentation (also normalization)
- Reuse metrics from previous exercise

In [2]:
m_path = '/home/tungnguyendinh/.fastai/data/pascal_2007/'
print(os.listdir(m_path))

#Define classes and encode/decode functions

VOC_CLASSES = (
    'aeroplane', 'bicycle', 'bird', 'boat',
    'bottle', 'bus', 'car', 'cat', 'chair',
    'cow', 'diningtable', 'dog', 'horse',
    'motorbike', 'person', 'pottedplant',
    'sheep', 'sofa', 'train', 'tvmonitor'
)

def encode_label(labels, classes = VOC_CLASSES):
    target = torch.zeros(len(classes))
    for l in labels:
        idx = classes.index(l)
        target[idx] = 1
    return target

def decode_target(target, threshold = 0.5, classes = VOC_CLASSES):
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
            result.append(classes[i])
    return ' '.join(result)

['test.json', 'valid.json', 'train.csv', 'test.csv', 'train.json', 'segmentation', '.ipynb_checkpoints', 'test', 'train']


In [3]:
from torchvision.io import read_image
#Create Custom dataset class for Pascal_2007
class PascalDataset(Dataset):
    def __init__(self, img_dir, annotations_file, transform=None, target_transform=None, is_train = False, is_valid = False):
        self.img_labels = pd.read_csv(annotations_file)
        if is_train:
            self.img_labels = self.img_labels[~self.img_labels["is_valid"]]
        if is_valid:
            self.img_labels = self.img_labels[self.img_labels["is_valid"]]
            
                
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1].split(" ")
        if self.transform:
            image = self.transform(image)
        return image, encode_label(label)


In [4]:
class PascalVOCDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir):
        super().__init__()
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.mean = [0.457342265910642, 0.4387686270106377, 0.4073427106250871]
        self.std = [0.26753769276329037, 0.2638145880487105, 0.2776826934044154]  
        self.train_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((300, 300)),
                transforms.RandomChoice([
                                        transforms.ColorJitter(brightness=(0.9, 1.1)),
                                        transforms.RandomGrayscale(p = 0.25)
                                        ]),
                transforms.RandomHorizontalFlip(p = 0.25),
                transforms.RandomRotation(25),
                transforms.ToTensor(), 
                transforms.Normalize(mean = self.mean, std = self.std),
                ])
        self.eval_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize((300, 300)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean = self.mean, std = self.std)
                   ])
        
    def setup(self):
        stage = "train"
        self.voc_train = PascalDataset(f"{self.data_dir}/{stage}", f"{self.data_dir}/{stage}.csv", transform=self.train_transform, is_train= True)
        self.voc_val = PascalDataset(f"{self.data_dir}/{stage}", f"{self.data_dir}/{stage}.csv", transform=self.eval_transform, is_valid = True) 
        stage = "test"                       
        self.voc_test = PascalDataset(f"{self.data_dir}/{stage}", f"{self.data_dir}/{stage}.csv", transform=self.eval_transform)
            
    def train_dataloader(self):
        self.setup()
        return DataLoader(self.voc_train, self.batch_size)
    
    def val_dataloader(self):
        return DataLoader(self.voc_val, self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.voc_test, self.batch_size)

In [5]:
BATCH_SIZE = 16
voc = PascalVOCDataModule(BATCH_SIZE, m_path)

In [6]:
from sklearn.metrics import accuracy_score
import numpy as np


def Accuracy_n_HammingScore(y_pred, y_true):
    y_pred = torch.round(y_pred)
    label = np.asarray(y_true.cpu())
    prob = np.asarray(y_pred.cpu())
    batch_size = label.shape[0]
    acc = 0
    hl = 0
    for i in range(batch_size):
        acc += accuracy_score(prob[i], label[i], normalize=True)
        hl += sum(torch.logical_and(
            y_true[i], y_pred[i])) / sum(torch.logical_or(y_true[i], y_pred[i]))
    acc /= batch_size
    hl /= batch_size
    acc = torch.Tensor([acc])
    return acc, hl

## Model Building 
Summary:
- Rebuild Model class (ResNet) with pytorch lighning module
- Make use of functions such as training_step, validation_step within LightningModule to organize training and validating processes
- Achieved similar (if not better) result with model from pure PyTorch
- Make use of recall to log metrics and filter visualization to TensorBoard, also load pretrained weights from previous exercise
- Use localhost:6007 on any browser to view TensorBoard logging

In [7]:
from pytorch_lightning.utilities.types import EPOCH_OUTPUT

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels,
                               kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(
            out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))

        x = self.relu(self.batch_norm2(self.conv2(x)))

        x = self.conv3(x)
        x = self.batch_norm3(x)

        # downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        # add identity
        x += identity
        x = self.relu(x)

        return x


class Block(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x.clone()

        x = self.relu(self.batch_norm2(self.conv1(x)))
        x = self.batch_norm2(self.conv2(x))

        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        x += identity
        x = self.relu(x)
        return


class ResNet(pl.LightningModule):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(
            num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(
            ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(
            ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(
            ResBlock, layer_list[3], planes=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
    
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []

        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion,
                          kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )

        layers.append(ResBlock(self.in_channels, planes,
                      i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion

        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=1e-3, momentum=0.9)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 12, eta_min=0, last_epoch=-1)
        return [optimizer], [scheduler]
    
    def training_step(self, train_batch, batch_idx):
        X, y = train_batch

        pred = self.forward(X)
        loss = self.loss_fn(pred, y)
        tmp_pred = nn.Sigmoid()(pred).detach()
        tmp_y    = y.detach()
        acc, hl = Accuracy_n_HammingScore(tmp_pred, tmp_y)
        batch_dictionary = {
            "loss" : loss,
            "acc"  : acc,
            "hl"   : hl
        }
        if (batch_idx % 50 == 0):    
            print(f"loss: {loss:>7f}, training accuracy: {100*acc.item():>0.1f}%, hamming score: {100*hl.item():>0.1f}% Batch: {batch_idx}")       
             
        return batch_dictionary
    
    def validation_step(self, val_batch, batch_idx):
        X, y = val_batch
        pred = self.forward(X)
        loss = self.loss_fn(pred, y)
        pred = nn.Sigmoid()(pred)
        acc, hl = Accuracy_n_HammingScore(pred, y)
        batch_dictionary = {
            "loss" : loss,
            "acc"  : acc,
            "hl"   : hl
        }
        return batch_dictionary
                    
    
def ResNet34(num_classes, channels=3):
    return ResNet(Block, [3, 4, 6, 3], num_classes, channels)

def ResNet50(num_classes, channels=3):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes, channels)



In [8]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import Callback

class ResNetCallback(Callback):
    def __init__(self):
        super().__init__()
        self.train_outputs = []
        self.val_outputs = []
    
    def on_fit_start(self, trainer, pl_module):
        pretrained_dict = torch.load('runs/ResNet50_Pretrained_Standardized_exp0/model.pth')
        model_dict = pl_module.state_dict()
        pretrained_dict = {k:v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        pl_module.load_state_dict(model_dict)
        
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, unused = 0):
        self.train_outputs.append(outputs)
        
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, unused=0):
        self.val_outputs.append(outputs)
    
    def on_train_epoch_end(self, trainer, pl_module):
        avg_loss = torch.stack([x['loss'] for x in self.train_outputs]).mean()
        avg_acc  = torch.stack([x['acc'] for x in self.train_outputs]).mean()
        avg_hl   = torch.stack([x['hl'] for x in self.train_outputs]).mean()
        
        pl_module.logger.experiment.add_scalar("Loss/train",
                                          avg_loss,
                                          pl_module.current_epoch)
        
        pl_module.logger.experiment.add_scalar("Accuracy/train",
                                          avg_acc,
                                          pl_module.current_epoch)
        
        pl_module.logger.experiment.add_scalar("HammingScore/train",
                                          avg_hl,
                                          pl_module.current_epoch)
            
    def on_validation_epoch_end(self, trainer, pl_module):
        avg_loss = torch.stack([x['loss'] for x in self.val_outputs]).mean()
        avg_acc  = torch.stack([x['acc'] for x in self.val_outputs]).mean()
        avg_hl   = torch.stack([x['hl'] for x in self.val_outputs]).mean()

        #Logging to tensorboard
        pl_module.logger.experiment.add_scalar("Loss/validation",
                                          avg_loss,
                                          pl_module.current_epoch)
        
        pl_module.logger.experiment.add_scalar("Accuracy/validation",
                                          avg_acc,
                                          pl_module.current_epoch)
        
        pl_module.logger.experiment.add_scalar("HammingScore/validation",
                                          avg_hl,
                                          pl_module.current_epoch)

In [9]:
model = ResNet50(20)
logger = TensorBoardLogger('runs', name='ResNet50_Pretrained_Lightning')
trainer = pl.Trainer(accelerator="gpu"
                     , max_epochs=1
                     , devices=[1]
                     , progress_bar_refresh_rate=1
                     , logger=logger
                     , callbacks=[ResNetCallback()]
                     )
trainer.fit(model, voc.train_dataloader(), voc.val_dataloader())

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

   | Name        | Type              | Params
---------------------------------------------------
0  | loss_fn     | BCEWithLogitsLoss | 0     
1  | conv1       | Conv2d            | 9.4 K 
2  | batch_norm1 | BatchNorm2d       | 128   
3  | relu        | ReLU              | 0     
4  | max_pool    | MaxPool2d         | 0     
5  | layer1  

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 314/314 [02:30<00:00,  2.08it/s, loss=0.184, v_num=9]


In [10]:
sampleImg = torch.rand((1,3,300,300))
logger.experiment.add_graph(model, sampleImg)

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/torch/_jit_internal.py:505: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  item = getattr(mod, name)
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/torch/_jit_internal.py:505: LightningDeprecationWarning: The `LightningModule.model_size` property was deprecated in v1.5 and will be removed in v1.7. Please use the `pytorch_lightning.utilities.memory.get_model_size_mb`.
  item = getattr(mod, name)


In [13]:
def makegrid(output, numrows):
    outer = (torch.Tensor.cpu(output).detach())
    plt.figure(figsize=(20,5))
    b = np.array([]).reshape(0, outer.shape[2])
    c = np.array([]).reshape(numrows*outer.shape[2], 0)
    i=0
    j=0
    while(i<outer.shape[1]):
        img = outer[0][i]
        b = np.concatenate((img,b), axis=0)
        j += 1
        if (j == numrows):
            c = np.concatenate((c,b), axis=1)
            b = np.array([]).reshape(0, outer.shape[2])
            j = 0
        i += 1
    return c

def ActivationMaps(pl_module, x, device = 1):
        pl_module.logger.experiment.add_image("input", torch.Tensor.cpu(x[0]), pl_module.current_epoch, dataformats="CHW")
        
        out = pl_module.relu(pl_module.batch_norm1(pl_module.conv1(x)))
        out = pl_module.max_pool(out)
        c = makegrid(out, 4)
        pl_module.logger.experiment.add_image("layer0", c, pl_module.current_epoch, dataformats="HW")
        
        out = pl_module.layer1(out)
        c = makegrid(out, 4)
        pl_module.logger.experiment.add_image("layer1", c, pl_module.current_epoch, dataformats="HW")
        
        out = pl_module.layer2(out)
        c = makegrid(out, 4)
        pl_module.logger.experiment.add_image("layer2", c, pl_module.current_epoch, dataformats="HW")

        out = pl_module.layer3(out)
        c = makegrid(out, 4)
        pl_module.logger.experiment.add_image("layer3", c, pl_module.current_epoch, dataformats="HW")

        out = pl_module.layer4(out)
        c = makegrid(out, 4)
        pl_module.logger.experiment.add_image("layer4", c, pl_module.current_epoch, dataformats="HW")


In [14]:
sample_img = next(iter(voc.train_dataloader()))[0]
ActivationMaps(model, sample_img)

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

<Figure size 2000x500 with 0 Axes>

In [15]:
! tensorboard --logdir=runs --port=6007

TensorFlow installation not found - running with reduced feature set.
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.13.0 at http://loc